In [18]:
print("Abhi")

Abhi


In [19]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone

from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import pinecone
from langchain.vectorstores import Pinecone as LangchainPinecone

In [20]:
PINECONE_API_KEY="pcsk_4ox7EQ_48i9bJpVkFXjr1qGS1xRptLkMiiMQtUTkxVEXiQDmohCXvBAfRNu7q9CtXvRTG5"
PINECONE_API_ENV="us-east-1-aws"

#extract data from the PDF

In [21]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

In [22]:
extracted_data=load_pdf("data/")

In [23]:
#extracted_data

create test chunks

In [24]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [25]:
text_chunks=text_split(extracted_data)
print("Length of my chunks:",len(text_chunks))

Length of my chunks: 5859


In [26]:
import sentence_transformers
print(sentence_transformers.__version__)


2.2.2


Download Embeddding model

In [27]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    return embeddings

In [28]:
embeddings=download_hugging_face_embeddings()

c:\Users\ASUS\anaconda3\envs\mchatbot\lib\site-packages\transformers\modeling_utils.py:461: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, 

In [29]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

venv name = mchatbot

In [30]:
query_result=embeddings.embed_query("hello world")
print("length", len(query_result))

length 384


In [31]:
query_result

[-0.03447732329368591,
 0.031023161485791206,
 0.006734949070960283,
 0.02610895223915577,
 -0.03936203941702843,
 -0.16030243039131165,
 0.06692401319742203,
 -0.006441463716328144,
 -0.04745052009820938,
 0.014758840203285217,
 0.07087533921003342,
 0.055527545511722565,
 0.019193334504961967,
 -0.026251288130879402,
 -0.010109501890838146,
 -0.026940416544675827,
 0.022307397797703743,
 -0.022226659581065178,
 -0.1496926099061966,
 -0.017493031919002533,
 0.007676285691559315,
 0.05435223504900932,
 0.003254441311582923,
 0.031725917011499405,
 -0.08462145179510117,
 -0.029405971989035606,
 0.05159560590982437,
 0.048124030232429504,
 -0.003314853413030505,
 -0.058279167860746384,
 0.041969235986471176,
 0.022210724651813507,
 0.128188818693161,
 -0.022338958457112312,
 -0.011656282469630241,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122604131698608,
 -0.03117535263299942,
 0.0526995025575161,
 0.047034818679094315,
 -0.0842030942440033,
 -0.030056176707148552,
 -0.0207448

In [32]:
from pinecone import Pinecone, ServerlessSpec, PodSpec # Make sure PodSpec is imported if you might use it

# Initialize Pinecone
# Replace with your actual API key and environment
# Ensure your environment matches the region you plan to use if it's region-specific
PINECONE_API_KEY = PINECONE_API_KEY
PINECONE_API_ENV = PINECONE_API_ENV # e.g., "us-east-1-aws" or similar, check your dashboard

pinecone = Pinecone(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

index_name = "medical-chatbot"

# Check if the index exists
# pinecone.list_indexes() returns a list of dictionaries, so we need to check names
existing_indexes = pinecone.list_indexes()
existing_index_names = [idx['name'] for idx in existing_indexes]

if index_name not in existing_index_names:
    print(f"Creating index '{index_name}'...")
    pinecone.create_index(
        name=index_name,
        dimension=1536,  # Ensure this matches your embedding model's dimension
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1') # Or your free-tier supported region
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists. Connecting to it.")

# Now you can connect to your index
index = pinecone.Index(index_name)

# You can also add a small check to ensure it's ready, especially after creation
# This is more relevant for Pod-based indexes, but good practice
# try:
#     # For Serverless, it should be ready almost immediately after creation call returns
#     # For Pod, you might need to loop and check index.describe_index_stats() until 'ready'
#     index.describe_index_stats()
#     print(f"Successfully connected to and can query index '{index_name}'.")
# except Exception as e:
#     print(f"Error connecting to index '{index_name}': {e}")

print(f"Index object: {index}")

Index 'medical-chatbot' already exists. Connecting to it.
Index object: <pinecone.data.index.Index object at 0x0000016DC9441C40>


In [33]:
from langchain_pinecone import Pinecone as LangChainPinecone # Alias LangChain's Pinecone class
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document # Import if your text_chunks are Document objects

# --- Assume these variables are already correctly defined and initialized from previous steps ---
# PINECONE_API_KEY
# PINECONE_API_ENV
# index_name
# text_chunks (e.g., list of Document objects with .page_content)
# embeddings (your HuggingFaceEmbeddings or OpenAIEmbeddings instance)
# ---------------------------------------------------------------------------------------------

import os

os.environ["PINECONE_API_KEY"] = "pcsk_4ox7EQ_48i9bJpVkFXjr1qGS1xRptLkMiiMQtUTkxVEXiQDmohCXvBAfRNu7q9CtXvRTG5"
os.environ["PINECONE_API_ENV"] = "your-env"  # e.g., "gcp-starter"


# Ensure the embedding model is initialized
# This is usually done once at the top level
# embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
# embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

index_name = "medical-chatbot"

# --- Create/Load Vectorstore using LangChain's Pinecone integration ---
print(f"Attempting to create/load Pinecone vectorstore for index '{index_name}' using LangChain...")

try:
    # Use LangChainPinecone (the aliased class from langchain_pinecone)
    # This class handles embedding the text_chunks and upserting them to Pinecone
    docsearch = LangChainPinecone.from_texts(
        [t.page_content for t in text_chunks], # Ensure this extracts pure strings if text_chunks are Document objects
        embeddings,
        index_name=index_name,
        # If your LangChainPinecone version requires it, you might also need to pass
        # api_key="pcsk_4ox7EQ_48i9bJpVkFXjr1qGS1xRptLkMiiMQtUTkxVEXiQDmohCXvBAfRNu7q9CtXvRTG5",
        # environment=PINECONE_API_ENV
        # However, it often picks these up from environment variables or the global pinecone.init()
        # for the latest versions of langchain-pinecone.
    )
    print("Pinecone vectorstore created/loaded successfully using LangChain.")

except Exception as e:
    print(f"ERROR: Failed to create/load Pinecone vectorstore with LangChain: {e}")
    # Provide more specific guidance for common issues
    print("Possible reasons:")
    print("1. 'text_chunks' format: Ensure it's a list of strings or list of Document objects with '.page_content'.")
    print("2. Environment/Kernel Issue: If 'AttributeError: from_texts...' persists, restart your Python kernel.")
    print("3. API Key/Environment: Double-check that PINECONE_API_KEY and PINECONE_API_ENV are correct and accessible.")
    print("4. Index Dimension Mismatch: The dimension of your embeddings must match the dimension of your Pinecone index.")
    raise # Re-raise the exception for full traceback

Attempting to create/load Pinecone vectorstore for index 'medical-chatbot' using LangChain...
Pinecone vectorstore created/loaded successfully using LangChain.


In [34]:
from langchain.vectorstores import Pinecone
from langchain_pinecone import Pinecone as LangChainPinecone

docsearch = LangChainPinecone.from_existing_index(
    index_name="medical-chatbot",
    embedding=embeddings
)

query="What are Allergies"

docs=docsearch.similarity_search(query,k=3)

print("Result:", docs)

Result: [Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(page_content='reacti

In [35]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [36]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [37]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [41]:
from langchain.vectorstores import Pinecone

# Wrap Pinecone index as LangChain VectorStore
vectorstore = Pinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,  # Must come from langchain.vectorstores.Pinecone
    return_source_documents=True,  # ✅ fixed typo
    chain_type_kwargs=chain_type_kwargs
)


In [ ]:
while True:
    user_input=input(f"Input Prompt: ")
    result=qa({"query":user_input})
    print("Response: ",result["result"])